![NYC Skyline](nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
This is a CSV file containing data on Airbnb listing prices and locations.
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

In [1]:
# Import necessary packages
import pandas as pd
import numpy as np

# Begin coding here ...
# Use as many cells as you like

In [2]:
"""
for file path accessibility and locating purpose
"""
import os 

os.getcwd()  # to retrieve current work directory 

os.chdir('...') # for file path changes purpose
# eg: /folder_x/subfolder/subfolder/....

os.listdir('...') # to list available folders at given file path 
# eg:  /folder_x/subfolder/subfolder/destination_path

['data', 'notebook.ipynb', 'nyc.jpg']

There are _3 file & folder paths_ that is accessible and available for further selection purpose, namely: 
- data (folder)
- notebook.ipynb 
- nyc.jpg (image file)

In [232]:
# pd.read_fwf('airbnb_price.csv')  

_read_fwf_ function of **pandas** is another file read methods that i newly discovered, but it isn't useful in the context of current data analysis 

In [3]:
airbnb_price = pd.read_csv(".../airbnb_price.csv")
airbnb_price.head()

,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"


It's detected that _price_ requires further data processing or refinement; thus, *re* was imported for data cleaning purpsose. 

After data cleaning, it's transformed into _numerical data_ for further data analysis.

**Remarks:** AI-assisted pair programming was adopted for faster code chunck generation. 

In [4]:
import re

# Define a function to extract the numeric value from the price string
def extract_price(price_str):
    # Use regular expression to match the numeric value
    match = re.search(r'(\d+(?:\.\d+)?)', price_str)
    if match:
        return float(match.group(0))
    else:
        return None

# Apply the function to the price column
airbnb_price['price'] = airbnb_price['price'].apply(extract_price)

# Rename the column to 'price($)'
airbnb_price = airbnb_price.rename(columns={'price': 'price_in_dollars'})

# Print the updated DataFrame
print(airbnb_price.head())

   listing_id  price_in_dollars                nbhood_full
0        2595             225.0         Manhattan, Midtown
1        3831              89.0     Brooklyn, Clinton Hill
2        5099             200.0     Manhattan, Murray Hill
3        5178              79.0  Manhattan, Hell's Kitchen
4        5238             150.0       Manhattan, Chinatown


Both _"listing_id"_ and _"price_in_dollars"_ shows numerical data types, thus, they are analyzable by **.desribe()** function. 

In [44]:
airbnb_price.describe()

,listing_id,price_in_dollars
count,2.520900e+04,25209.000000
mean,2.068922e+07,141.777936
std,1.102928e+07,147.349137
min,2.595000e+03,0.000000
25%,1.202273e+07,69.000000
50%,2.234391e+07,105.000000
75%,3.037669e+07,175.000000
max,3.645581e+07,7500.000000


There are `25,209` data points available in _airbnb_price_ dataset, it's discovered that the minimum hotel room rate can start from as low as $0. 

The price variability of high end and average hotel room rate is quite huge. 
Nonetheless, the average charging rate available from this agoda dataset was `$141.78` with standard deviation (most common price variability range) of `$147.35`. 

In [6]:
# Print data types of each column in DataFrame
print("\n")
print(airbnb_price.dtypes)



listing_id            int64
price_in_dollars    float64
nbhood_full          object
dtype: object


**Given Enquiry #1:** What is the average listing price? Round to the nearest two decimal places and save into a variable. <br>
<br>
**Answer:** $141.78

In [11]:
# Calculate the average of the "price($)" column
average_price = airbnb_price['price_in_dollars'].mean()

# Print the result
print(f'The average price is: ${average_price:.2f}')

The average price is: $141.78


This can be another methods to answer similar enquiry, depends on the exact context and data requirement assigned by business team representative. 

In [46]:
# Group by nbhood_full and calculate the average, min, and max prices
nbhood_airbnb_price = airbnb_price.groupby('nbhood_full').agg(
    AVG_PRICE=('price_in_dollars', 'mean'),
    MIN_PRICE=('price_in_dollars', 'min'),
    MAX_PRICE=('price_in_dollars', 'max')
).reset_index()

nbhood_airbnb_price

,nbhood_full,AVG_PRICE,MIN_PRICE,MAX_PRICE
0,"Bronx, Allerton",98.933333,35.0,450.0
1,"Bronx, Baychester",78.600000,53.0,101.0
2,"Bronx, Belmont",91.875000,24.0,299.0
3,"Bronx, Bronxdale",53.600000,40.0,79.0
4,"Bronx, Castle Hill",74.000000,62.0,86.0
...,...,...,...,...
212,"Staten Island, Todt Hill",82.333333,50.0,135.0
213,"Staten Island, Tompkinsville",78.848485,30.0,245.0
214,"Staten Island, Tottenville",160.800000,70.0,299.0
215,"Staten Island, West Brighton",67.142857,40.0,125.0


Let's move on to answer and resolve another data analysis requirement from **airbrb_room_type** dataset. 

In [43]:
airbnb_room_type = pd.read_excel("/work/files/workspace/data/airbnb_room_type.xlsx")
airbnb_room_type.head()

,listing_id,description,room_type
0,2595,Skylit Midtown Castle,Entire home/apt
1,3831,Cozy Entire Floor of Brownstone,Entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
3,5178,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt


It's discovered that _room_type_ data attribute requires data processing for data cleaning prior further data analysis. 

In [19]:
select distinct room_type 
from airbnb_room_type;

,room_type
0,private room
1,PRIVATE ROOM
2,ENTIRE HOME/APT
3,SHARED ROOM
4,Entire home/apt
5,Private room
6,entire home/apt
7,shared room
8,Shared room


`standardize_room_type` function was created to transform it into defined format as: 
- Private Room 
- Shared Room 
- Entire Home/Apt 

**Remarks:** 
- AI-assisted pair programming was adopted for faster code chunck generation. 
- The available _room_type_ attributes can be smarter and even lesser since it's transformed into lower case before being processed accordingly. 

In [20]:
# Define a function to standardize the room type
def standardize_room_type(room_type):
    room_type = room_type.strip().lower()
    if room_type in ['private room', 'private', 'privateroom']:
        return 'Private Room'
    elif room_type in ['shared room', 'shared room', 'SHARED ROOM']:
        return 'Shared Room'
    elif room_type in ['entire home/apt', 'entire home apt', 'entirehome/apt']:
        return 'Entire Home/Apt'
    else:
        return room_type

# Apply the function to the room_type column
airbnb_room_type['room_type'] = airbnb_room_type['room_type'].apply(standardize_room_type)

# Print the updated DataFrame
print(airbnb_room_type)

       listing_id                                 description        room_type
0            2595                       Skylit Midtown Castle  Entire Home/Apt
1            3831             Cozy Entire Floor of Brownstone  Entire Home/Apt
2            5099   Large Cozy 1 BR Apartment In Midtown East  Entire Home/Apt
3            5178             Large Furnished Room Near B'way     Private Room
4            5238          Cute & Cozy Lower East Side 1 bdrm  Entire Home/Apt
...           ...                                         ...              ...
25204    36425863  Lovely Privet Bedroom with Privet Restroom     Private Room
25205    36427429                    No.2 with queen size bed     Private Room
25206    36438336                             Seas The Moment     Private Room
25207    36442252               1B-1B apartment near by Metro  Entire Home/Apt
25208    36455809     Cozy Private Room in Bushwick, Brooklyn     Private Room

[25209 rows x 3 columns]


The _room_type_ attribute was cleansed accordingly, by using SQL query before attempting to provide feedback to given data analysis task #2: 

In [21]:
select distinct room_type 
from airbnb_room_type;

,room_type
0,Private Room
1,Shared Room
2,Entire Home/Apt


**Given Enquiry #2:**  How many of the listings are private rooms? Save this into any variable. <br>
**Answer**: 11356

In [22]:
select count(*) as nb_private_rooms
from airbnb_room_type
where room_type = 'Private Room';

,nb_private_rooms
0,11356


Up until now, it's discovered that pandas can analyze data from multiple formats, be it: 
- CSV 
- TSV 
- XLSX
<br> 

Let's proceed to analyze **airbnb_last_review.tsv** using **pd.read_csv()** function from _pandas_

In [48]:
airbnb_last_review = pd.read_csv(".../airbnb_last_review.tsv", sep='\t')
airbnb_last_review.head()

,listing_id,host_name,last_review
0,2595,Jennifer,May 21 2019
1,3831,LisaRoxanne,July 05 2019
2,5099,Chris,June 22 2019
3,5178,Shunichi,June 24 2019
4,5238,Ben,June 09 2019


It's discovered that **last_review** attribute requires data type adjustment, so that it can be changed from _text_ data_type into _date time_ data_type. 
<br> 
Since it's original data attribute was in Month-Day-Year format; thus, it's transformed into YYYY-MM-DD format to ensure data validity (as in, preprocess them without changing their original data meaning).
<br>

In [24]:
airbnb_last_review['last_review'] = pd.to_datetime(airbnb_last_review['last_review'] )
airbnb_last_review['last_review'] = airbnb_last_review['last_review'].dt.strftime("%m-%d-%Y")
airbnb_last_review['last_review'] = pd.to_datetime(airbnb_last_review['last_review']).dt.strftime("%Y-%m-%d")
airbnb_last_review.head()

,listing_id,host_name,last_review
0,2595,Jennifer,2019-05-21
1,3831,LisaRoxanne,2019-07-05
2,5099,Chris,2019-06-22
3,5178,Shunichi,2019-06-24
4,5238,Ben,2019-06-09


In [25]:
airbnb_last_review.describe(include='all')

,listing_id,host_name,last_review
count,2.520900e+04,25201,25209
unique,NaN,7363,190
top,NaN,Michael,2019-06-23
freq,NaN,215,1413
mean,2.068922e+07,NaN,NaN
std,1.102928e+07,NaN,NaN
min,2.595000e+03,NaN,NaN
25%,1.202273e+07,NaN,NaN
50%,2.234391e+07,NaN,NaN
75%,3.037669e+07,NaN,NaN


It's discovered that **.describe()** function from _pandas_ aren't practical enough for _date_ data_type. 

**Given Enquiry #3:** <br>
What are the dates of the earliest and most recent reviews? Store these values as two separate variables with your preferred names.<br>
<br>
**Answer:** <br>
The **earliest** review date was received on **01 Jan 2019**. <br>
The **latest** review date was received on **09 July 2019**. 

In [26]:
select min(last_review) as first_reviewed, max(last_review) as last_reviewed
from airbnb_last_review;


,first_reviewed,last_reviewed
0,2019-01-01,2019-07-09


Before resolving the last assigned data analysis tasks at hand, null values analysis was initiated. 

In [28]:
print(airbnb_price.isnull().sum())
print()
print(airbnb_room_type.isnull().sum())
print()
print(airbnb_last_review.isnull().sum())

listing_id          0
price_in_dollars    0
nbhood_full         0
dtype: int64

listing_id      0
description    10
room_type       0
dtype: int64

listing_id     0
host_name      8
last_review    0
dtype: int64


It's discovered that there are _null_ values at the **description** attributes from _airbnb_room_type_ dataset.
<br>
Thus, there are replaced with "N/A" values. 

In [31]:
# Replace null values with "N/A" in the <description> column
airbnb_room_type['description'] = airbnb_room_type['description'].fillna('N/A')

In [29]:
print(airbnb_price.describe(include='all'))

          listing_id  price_in_dollars                   nbhood_full
count   2.520900e+04      25209.000000                         25209
unique           NaN               NaN                           217
top              NaN               NaN  Brooklyn, Bedford-Stuyvesant
freq             NaN               NaN                          2209
mean    2.068922e+07        141.777936                           NaN
std     1.102928e+07        147.349137                           NaN
min     2.595000e+03          0.000000                           NaN
25%     1.202273e+07         69.000000                           NaN
50%     2.234391e+07        105.000000                           NaN
75%     3.037669e+07        175.000000                           NaN
max     3.645581e+07       7500.000000                           NaN


**Given Enquiry #4:** <br>
Combine the new variables into one DataFrame called review_dates with four columns in the following order: 
- first_reviewed
- last_reviewed
- nb_private_rooms
- avg_price
<br> 
The DataFrame should only contain one row of values.  
<br>
**Finding / Result:** <br>


In [35]:
review_dates = pd.concat([df2, df6, avg_price], axis=1)
review_dates

,first_reviewed,last_reviewed,nb_private_rooms,avg_price
0,2019-01-01,2019-07-09,11356,141.78


**Lesson Learned from this task:** 
- The adherance of data confidentiality was reinforced and practiced in this exploration. 
- Though data analysts might got accessibility to more data for information exploration, but it's shouldn't be executed if there is no assigned tasks explicitly requires accessibility or analysis upon it. 
- So, how much trust one business entity given to their employees? May be this is one of the many reason(s) why there are zero-trust was practiced. 